# 8: Building Your 3D Vision Pipeline

As we now are comming to an end of this series, we have covered a lot of ground. We have learned about the basics of 3D vision, the first half we covered a singel camera; How to model a camera, how to project 3D points into 2D and how to estimate a pose from 2D-3D correspondences. In the second half we covered stereo vision; How to model two cameras compared to each other, how to triangulate 3D points from two 2D points, epipolar geometry and stereo matching.

For the final workshop we will try to put the tools we have made into a full 3D stereo matching pipeline.

## 3D Stereo Matching Pipeline

To make a complete pipeline we will combine from all the previous workshops. We start by loading test data, 

:::{admonition} Dataset
:class: tip, dropdown
In this workshop we will use the Middlebury Stereo Evaluation dataset. The dataset can be seen here:

- [2021 Mobile stereo datasets with ground truth](https://vision.middlebury.edu/stereo/data/scenes2021/)
- [Download all as zip](https://vision.middlebury.edu/stereo/data/scenes2021/zip/all.zip)

Only the dataset "traproom1" is included in this repository. You can download the rest using the zip link above. To make it seamlessly integrate with repo by:

- Placing the desired scene folders in the `test_data` folder.
- Add the dataset to `TestDataPaths` similar to that of `traproom1`:
    ```python
    class TestDataPaths:
        ...
        traproom1_dir: Path = _test_data_dir / "traproom1"
        your_folder_dir: Path = _test_data_dir / "your_folder"
    ```
:::

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


from oaf_vision_3d._notebook_tools import simple_plot
from oaf_vision_3d._test_data_paths import TestDataPaths
from oaf_vision_3d.block_matching import block_matching
from oaf_vision_3d.plane_sweeping import plane_sweeping
from oaf_vision_3d.point_cloud_visualization import open3d_visualize_point_cloud
from oaf_vision_3d._stereo_data_reader import StereoData
from oaf_vision_3d.triangulation import triangulate_disparity

In [ ]:
data_dir = TestDataPaths.traproom1_dir
stereo_data = StereoData.from_path(data_dir)
rgb = stereo_data.image_0

fig, ax = plt.subplots(2, 1, figsize=(12, 12))
ax[0].imshow(stereo_data.image_0, cmap="gray", vmin=0, vmax=1)
ax[0].set_title("Image left")
ax[0].axis("off")
ax[1].imshow(stereo_data.image_1, cmap="gray", vmin=0, vmax=1)
ax[1].set_title("Image right")
ax[1].axis("off")
plt.tight_layout()
plt.show()

### Block Matching

We can perform block matching on the stereo images to get a disparity map using the [`block_matching`](../oaf_vision_3d/block_matching.py) module that we implemented in [workshop 6](./06_stereo_matching_fundamentals.ipynb).

In [ ]:
disparity_map = block_matching(
    image_0=stereo_data.image_0,
    image_1=stereo_data.image_1,
    disparity_range=stereo_data.expected_disparity,
    block_size=np.array([11, 11], dtype=np.int32),
)
simple_plot(data=disparity_map, title="Block Matching Disparity Map")

From the dispariries we can triangualte a depth map using the triangulation math from [workshop 5](./05_dual_camera_setups.ipynb). The function [`triangulate_disparity`] that we made in [workshop 6](./06_stereo_matching_fundamentals.ipynb) can be used to triangulate the disparity map:

In [ ]:
xyz_block_mathcing = triangulate_disparity(
    disparity=disparity_map,
    lens_model_0=stereo_data.lens_model_0,
    lens_model_1=stereo_data.lens_model_1,
    transformation_matrix=stereo_data.transformation_matrix,
)
simple_plot(data=xyz_block_mathcing[..., 2], title="Block Matching Depth Map")

In [ ]:
open3d_visualize_point_cloud(xyz=xyz_block_mathcing, rgb=rgb)

### Plane Sweeping

In [workshop 7](./07_stereo_matching_fundamentals_continued.ipynb) we looked at how plane sweeping as an alternative way to do block matching. We can use the function `plane_sweeping` from the [`plane_sweeping`](../oaf_vision_3d/plane_sweeping.py) module to get the xyz values directly:

In [ ]:
xyz_plane_sweeping = plane_sweeping(
    image=stereo_data.image_0,
    lens_model=stereo_data.lens_model_0,
    secondary_images=[stereo_data.image_1],
    secondary_lens_models=[stereo_data.lens_model_1],
    secondary_transformation_matrices=[stereo_data.transformation_matrix],
    depth_range=np.array([5000, 20000], dtype=np.float32),
    step_size=150.0,
    block_size=np.array([11, 11], dtype=np.int32),
)
simple_plot(data=xyz_plane_sweeping[..., 2], title="Plane Sweeping Depth Map")

In [ ]:
open3d_visualize_point_cloud(xyz=xyz_plane_sweeping, rgb=rgb)

An added advantage of plane sweeping is that it does not limit us to a stereo camera setup, in fact, we can use any number of cameras to get the depth map. This uses the constrains given by the calibration of the cameras to match agains all images at the same time. We can show this of on the following dataset:

In [ ]:
stereo_data_0 = StereoData.from_path(TestDataPaths.stereo_data_0_dir)
stereo_data_1 = StereoData.from_path(TestDataPaths.stereo_data_1_dir)

fig, ax = plt.subplots(3, 1, figsize=(12, 15))
ax[0].imshow(stereo_data_1.image_1, cmap="gray", vmin=0, vmax=1)
ax[0].set_title("Image left")
ax[0].axis("off")
ax[1].imshow(stereo_data_0.image_0, cmap="gray", vmin=0, vmax=1)
ax[1].set_title("Image main")
ax[1].axis("off")
ax[2].imshow(stereo_data_0.image_1, cmap="gray", vmin=0, vmax=1)
ax[2].set_title("Image right")
ax[2].axis("off")
plt.tight_layout()
plt.show()

We run the same plane sweeping algorithm, but now giving both the left and right images as secondary images:

In [ ]:
xyz_plane_sweeping_multiple = plane_sweeping(
    image=stereo_data_0.image_0,
    lens_model=stereo_data_0.lens_model_0,
    secondary_images=[stereo_data_0.image_1, stereo_data_1.image_1],
    secondary_lens_models=[stereo_data_0.lens_model_1, stereo_data_1.lens_model_1],
    secondary_transformation_matrices=[
        stereo_data_0.transformation_matrix,
        stereo_data_1.transformation_matrix,
    ],
    depth_range=np.array([135.0, 142.0], dtype=np.float32),
    step_size=0.5,
    block_size=np.array([11, 11], dtype=np.int32),
)
simple_plot(
    data=xyz_plane_sweeping_multiple[..., 2],
    title="Multiple Image Plane Sweeping Depth Map",
)

In [ ]:
open3d_visualize_point_cloud(xyz=xyz_plane_sweeping_multiple, rgb=stereo_data_0.image_0)

### Conclusion

In this workshop we have combined all the tools we have made in this series to make a full 3D stereo matching pipeline. The results are not perfect, but they are a good starting point for further development. Even though we can cleary see the limitations with the block matching algorithm, the undelying math is close to same for most baseline based triangulation methods. Meaning the knowledge we have gained in this series can be used to understand and develop more advanced 3D vision algorithms.

## Deliverables

The website hosted by the repository is currently owned and published by the repository owner, but it is highly likely that this series will be refined as I move forward. To pass this project you thus have to fork and host the website yourself. This makes sure you keep a version of this project, at the state it was when you passed the project. It also makes sure you have a place to show your work to future employers.

To host the website yourself you have to follow the instuctions below. These instructions are for forking the `main` branch of the repository, if you have local changes you want to keep you can add these changes to the PR below. I would be happy to act as a reviewer for your PR.

### 1. Fork the Repository

This step will create a copy of the repository in your GitHub account:

1. Navigate to the [repository](https://github.com/martvald/oaf-3d-vision-pipeline-workshop).
2. Click the **`Fork`** button in the top-right corner of the page.
3. Enter a name for your forked repository (e.g., `pixel-to-point-cloud`) and optionally add a description.

   ![Fork repository](../test_data/forking/fork_repository.png)

4. You’ll be redirected to your fork. If not, visit your GitHub profile and click on the newly forked repository.


### 2. Configure GitHub Pages

We need to set up GitHub Pages for your forked repository to host the website:

1. In your forked repository, go to the **`Settings`** tab.
2. Scroll to the **`Pages`** section.
3. Select **GitHub Actions** as the source.
4. (Optional) Set up a custom domain. By default, your site will be available at:

    ```bash
    https://<your-username>.github.io/<your-repo-name>/
    ```

    ![Configure GitHub Pages](../test_data/forking/configure_github_pages.png)

### 3. Enable GitHub Actions Workflows

To publish the website, we need to use the GitHub Actions workflow in the repository. These are disabled by default in forked repositories:

1. Navigate to the **`Actions`** tab.
2. Since workflows are disabled by default in forked repositories, enable them by clicking: **"I understand my workflows, go ahead and enable them."**
   
   ![Enable workflows](../test_data/forking/enable_github_actions.png)

### 4. Make Changes and Create a Pull Request (PR)

Now everything is set up for you to make changes to the repository and publish them. The forked repo has some paths that should be updated so the links work correctly. To do this, follow the steps below:

#### a. Set Up the Repository Locally

1. Clone your forked repository to your local machine:

    ```bash
    git clone https://github.com/<your-username>/<your-repo-name>.git
    ```

2. Create a new branch for your changes:

    ```bash
    git checkout -b workshop-delivery
    ```

#### b. Update Repository Links

1. Update the following files:

   - `README.md`:

      Replace the link to the original site:

      ```markdown
      https://teampurpler.github.io/oaf-3d-vision-pipeline-workshop/
      ```

      with your GitHub Pages link:

      ```markdown
      https://<your-username>.github.io/<your-repo-name>/
      ```

   - `landing.py`:

      Replace all instances of:

      ``` bash
      teampurpler/oaf-3d-vision-pipeline-workshop
      ```

      with your user and repository name:

      ``` bash
      <your-username>/<your-repo-name>
      ```

   - `workshops/08_building_your_3d_vision_pipeline.ipynb`:

      This is the script containing the current tutorial on forking the repository. You can update this as well if you to show how to fork from you repository. To do this replace all instances of:

      ``` bash
      martvald
      ```

      with your user name:

      ``` bash
      <your-username>
      ```

      and all instances of:

      ``` bash
      oaf-3d-vision-pipeline-workshop
      ```

      with your repository name:

      ``` bash
      <your-repo-name>
      ```

2. Commit and push your changes:

   ```bash
   git commit -am "Update links to my own repository"
   git push origin workshop-delivery
   ```

3. Optionally, you can make additional changes to the website content to include your own project changes. You can also add changes in later PRs and the website will update automatically.

3. Open a Pull Request (PR) from your `workshop-delivery` branch to the `main` branch of your forked repository.
   - Ensure the target is your fork, not the original repository.

### 5. Publish Your Changes

1. Wait for the GitHub Actions workflow to complete on your PR.
   - **No branch protection rules are applied by default, so there is no protection against merging the PR before the workflows finish.**
2. Review the generated static site:
   - Go to the **`Actions`** tab and select the documentation job.
   - Under **`Summary`**, download the artifact containing the static HTML files.
   - Open the `index.html` file locally to preview the site.
3. If everything looks good, merge the PR into the `main` branch.
4. The publish job will automatically deploy your changes to GitHub Pages.
5. Once the job finishes, visit your live site at:

   ``` bash
   https://<your-username>.github.io/<your-repo-name>/
   ```

When deployed successfully, you should be able to see the status in **`Pages`** section of your repository **`Settings`**:

![Deployed site](../test_data/forking/publised_pages.png)

### 6. Submit Your Project

Once you have successfully hosted the website, submit the link to me. I will review the website and the changes you have made to the repository. If everything looks good, I will pass you on the project.

## Next Steps

This is the final workshop in the series. I hope you have learned a lot and that you have enjoyed the journey. If you have any questions or feedback, feel free to reach out to me. I would love to hear from you.

Good luck with your future projects and I hope to see you again in future workshops.